In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

In [2]:
base_url = 'https://www.beckershospitalreview.com/finance/49-hospitals-furloughing-workers-in-response-to-covid-19.html'
response = requests.get(base_url)
soup = BeautifulSoup(response.content, 'html.parser')

In [106]:
pars = soup.find_all('p')
dict_list = []
date = 'None'
for par in pars[7:]:
    link = 'None'
    inner_date = 'None'
    state = 'Unspecified'
    index = par.text.split('.')[0]
    if is_month(index):
        date = par.strong.text
        pass
    if index.isnumeric():
        furloughs = find_furloughs(par)
        layoffs = find_layoffs(par)
        if par.strong:
            location = par.strong.text
        if par.a:
            link = par.a['href']
        words = par.text.split(' ')
        word_index= 0 
        for word in words:
            try:
                next_word = words[word_index+1]
            except IndexError:
                next_word = 'None'
            try:
                previous_word = words[word_index-1]
            except IndexError:
                previous_word = 'None'
            if is_month(word):
                if next_word.isnumeric():
                    inner_date = word + ' ' + words[word_index+1]
#                     print(inner_date)
            elif is_state(word) != False:
                state = is_state(word)
            elif 'based' in word:
                new = word.replace('based','').replace('-','')
                if is_state(new)!= False:
                    state = is_state(new)
            word_index += 1
        new_dict = {
            'Date': date,
            'Furloughs': furloughs,
            'Layoffs': layoffs,
            'Location': location,
            'Link': link,
            'State': state
            }
        if inner_date != 'None':
            new_dict['Date']=inner_date
        dict_list.append(new_dict)
    return dict_list

In [110]:
list(pd.DataFrame(dict_list).columns)

['Date', 'Furloughs', 'Layoffs', 'Location', 'Link', 'State']

In [100]:
for par in pars[7:]:
    print(find_furloughs(par))
    print('')

Unspecified

1,100

1,000

1,500

300

175

200

None

None

None

None

Unspecified

2,600

1,200

None

211

150

100

70

30

None

Unspecified

2,800

1,100

219

60

None

None

None

4,100

None

Unspecified

None

30,000

Unspecified

2,475

1,000

None

None

200

125

None

None

None

None

None

None

None

None

Unspecified

None

170

1,300

422

None

52

Unspecified

None

650

400

200

52

25

None

None

None

None

Unspecified

None

60

None

None

None

None

None

None

54

None

Unspecified

None

None

Unspecified

Unspecified

None

600

650

None

500

360

50

None

80

Unspecified

None

Unspecified

None

None

Unspecified

500

480

630

150

None

75

Unspecified

None

50

50

50

None

None

None

Unspecified

600

Unspecified

None

700

337

125

None

Unspecified

None

99

67

None

Unspecified

100

283

400

400

None

None

None

Unspecified

None

None

None

None

Unspecified

None

None

Unspecified

900

300

None

Unspecified

None

None

Un

In [105]:
def find_furloughs(paragraph):
    sentences = paragraph.text.split('. ')
    for sentence in sentences:
        if 'furlough' in sentence:
            words = sentence.split(' ')
            word_index = 0
            number = 'Unspecified'
            number_index = 0
            for word in words:
                if 'furlough' in word:
                    furlough_index = word_index
                try:
                    next_word = words[word_index+1]
                except IndexError:
                    next_word = 'None'
                try:
                    previous_word = words[word_index-1]
                except IndexError:
                    previous_word = 'None'
                if word.replace(',','').replace('.','').isnumeric() and is_worker(next_word):
                    if number == 'Unspecified':
                        number = word
                        number_index = word_index
                    elif abs(number_index - furlough_index) > abs(word_index - furlough_index):
                        number = word
                        number_index = word_index
                word_index+=1
            if number != 'Unspecified':
                return number
    return 'Unspecified'

In [104]:
def find_layoffs(paragraph):
    sentences = paragraph.text.split('. ')
    for sentence in sentences:
        if 'lay' in sentence and 'off' in sentence:
            word_index=0
            number = 'Unspecified'
            number_index = 0
            words = sentence.split(' ')
            for word in words:
                if 'off' in word:
                    layoff_index = word_index
                try:
                    next_word = words[word_index+1]
                except IndexError:
                    next_word = 'None'
                try:
                    previous_word = words[word_index-1]
                except IndexError:
                    previous_word = 'None'
                if word.replace(',','').replace('.','').isnumeric() and is_worker(next_word):
                    if number == 'None':
                        number = word
                        number_index = word_index
#                         print(word + ' '+ 'layoffs')
                    elif abs(number_index - layoff_index) > abs(word_index - layoff_index):
                        number = word
                        number_index = word_index
                word_index+=1
            if number != 'Unspecified':
                return number
    return 'Unspecified'

In [94]:
worker_words = [
    'staffers',
    'employees',
    'workers',
    'people',
    'individuals',
    'caregivers',
    'of',
    'nonclinical',
    'staff'
]

In [4]:
def is_worker(word):
    for worker_word in worker_words:
        if worker_word in word:
            return True
    else:
        return False

In [5]:
months = [
    'January',
    'February',
    'March',
    'April',
    'May',
    'June',
    'July',
    'August',
    'September',
    'October',
    'November',
    'December'
]

In [6]:
def is_month(string):
    for month in months:
        if month in string:
            return True
    else:
        return False

In [7]:
states = ["AL", "AK", "AZ", "AR", "CA", "CO", "CT", "DC", "DE", "FL", "GA", 
          "HI", "ID", "IL", "IN", "IA", "KS", "KY", "LA", "ME", "MD", 
          "MA", "MI", "MN", "MS", "MO", "MT", "NE", "NV", "NH", "NJ", 
          "NM", "NY", "NC", "ND", "OH", "OK", "OR", "PA", "RI", "SC", 
          "SD", "TN", "TX", "UT", "VT", "VA", "WA", "WV", "WI", "WY", "Alabama", "Alaska", "Arizona", "Arkansas", "California", "Colorado",
              "Connecticut", "Delaware", "Florida", "Georgia", "Hawaii", "Idaho", "Illinois",
              "Indiana", "Iowa", "Kansas", "Kentucky", "Louisiana", "Maine", "Maryland",
              "Massachusetts", "Michigan", "Minnesota", "Mississippi", "Missouri", "Montana",
              "Nebraska", "Nevada", "New Hampshire", "New Jersey", "New Mexico", "New York",
              "North Carolina", "North Dakota", "Ohio", "Oklahoma", "Oregon", "Pennsylvania",
              "Rhode Island", "South Carolina", "South Dakota", "Tennessee", "Texas", "Utah",
              "Puerto Rico", "Vermont", "Virginia", "Washington", "West Virginia", "Wisconsin",
              "Wyoming"]

In [30]:
def is_state(word):
    word = word.replace('.','').replace('(','').replace(')','')
    for state in states:
        if state[0:3] == word[0:3]:
            return convert_state(state)
        elif state == word:
            return convert_state(state)
        elif state.title() == word:
            return convert_state(state)
    else:
        return False

In [9]:
us_state_abbrev = {
    'Alabama': 'AL',
    'Alaska': 'AK',
    'American Samoa': 'AS',
    'Arizona': 'AZ',
    'Arkansas': 'AR',
    'California': 'CA',
    'Colorado': 'CO',
    'Connecticut': 'CT',
    'Delaware': 'DE',
    'District of Columbia': 'DC',
    'Florida': 'FL',
    'Georgia': 'GA',
    'Guam': 'GU',
    'Hawaii': 'HI',
    'Idaho': 'ID',
    'Illinois': 'IL',
    'Indiana': 'IN',
    'Iowa': 'IA',
    'Kansas': 'KS',
    'Kentucky': 'KY',
    'Louisiana': 'LA',
    'Maine': 'ME',
    'Maryland': 'MD',
    'Massachusetts': 'MA',
    'Michigan': 'MI',
    'Minnesota': 'MN',
    'Mississippi': 'MS',
    'Missouri': 'MO',
    'Montana': 'MT',
    'Nebraska': 'NE',
    'Nevada': 'NV',
    'New Hampshire': 'NH',
    'New Jersey': 'NJ',
    'New Mexico': 'NM',
    'New York': 'NY',
    'North Carolina': 'NC',
    'North Dakota': 'ND',
    'Northern Mariana Islands':'MP',
    'Ohio': 'OH',
    'Oklahoma': 'OK',
    'Oregon': 'OR',
    'Pennsylvania': 'PA',
    'Puerto Rico': 'PR',
    'Rhode Island': 'RI',
    'South Carolina': 'SC',
    'South Dakota': 'SD',
    'Tennessee': 'TN',
    'Texas': 'TX',
    'Utah': 'UT',
    'Vermont': 'VT',
    'Virgin Islands': 'VI',
    'Virginia': 'VA',
    'Washington': 'WA',
    'West Virginia': 'WV',
    'Wisconsin': 'WI',
    'Wyoming': 'WY'
}

In [12]:
def convert_state(state):
    if state in us_state_abbrev.keys():
        state = us_state_abbrev[state]
    return state

In [108]:
convert_state(('Illinois'))

'IL'